In [2]:
import numpy as np
from pulp import *
import matplotlib.pyplot as plt

In [13]:
c = np.array([600, 400])
A = np.array([[2, 0.25], [4, 2], [3,1], [1,4]])
b = np.array([45, 100, 300, 50])

x = LpVariable.dicts("x", range(2), lowBound=0, cat='Continuous')
prob = LpProblem("problem", LpMaximize)
prob += lpSum([c[i]*x[i] for i in range(2)])
for i in range(len(A)):
    prob += lpSum([A[i][j]*x[j] for j in range(2)]) <= b[i]
    
prob.solve(PULP_CBC_CMD(msg=0))

print("Status:", LpStatus[prob.status])
print("Objective value:", value(prob.objective))
print("Solution:")
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Dual variables:")
for name, c in prob.constraints.items():
    print(name, "=", c.pi)
    
print(listSolvers(onlyAvailable=True))

Status: Optimal
Objective value: 15714.285440000001
Solution:
x_0 = 21.428571
x_1 = 7.1428571
Dual variables:
_C1 = -0.0
_C2 = 142.85714
_C3 = -0.0
_C4 = 28.571429
['PULP_CBC_CMD']
